In [ ]:
!pip install selenium
!apt-get -q update # to update ubuntu to correctly run apt install
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
chromium-chromedriver i

In [ ]:
#Importing packages
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
import pandas as pd

In [ ]:
#load in driver
driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [ ]:
driver.get('https://www.beeradvocate.com/beer/top-rated/')

In [ ]:
%%time

# create beer list for web scraper
beer_list = []

x = 2
while x < 252:
    beer = driver.find_elements_by_xpath("//*[@id='ba-content']/table/tbody/tr[{}]/td[2]/a/b".format(x))[0].text
    beer_list.append(beer)
    x += 1

CPU times: user 562 ms, sys: 26.7 ms, total: 589 ms
Wall time: 10.6 s


In [ ]:
len(beer_list)

250

In [ ]:
beer_list2 = beer_list[0:2]

//*[@id="rating_fullview_content_2"]/span[2]

In [ ]:
%%time
all_comments = pd.DataFrame(columns = ['Product_name','product_review', 'user_ratings', "overall_rating"]) 

page = 0

for beer in beer_list:
    print(page)
    
    driver.find_element_by_link_text("{}".format(beer)).click()
    driver.implicitly_wait(1)
     
    ids = driver.find_elements_by_xpath("//*[@id='rating_fullview_user']/div/a")
    
    rating = 0
    
    # not all beers have 25 reviews
    while rating <= (len(ids) - 1):

        product = beer
        
        # this gets the whole review with ratings and date, cant seem to figure out how to just get review
        review = driver.find_elements_by_xpath("//*[@id='rating_fullview_content_2']")[rating].text
        
        # this gets all of the ratings, dont exactly know which rating he wants 
        user_ratings = driver.find_elements_by_xpath("//*[@id='rating_fullview_content_2']/span[4]")[rating].text
        
        # this gets the big overall rating, im guessing this is what rating he wants us to use
        overall_rating = driver.find_elements_by_xpath("//*[@id='rating_fullview_content_2']/span[2]")[rating].text

        all_comments.loc[len(all_comments)] = [product,review,user_ratings,overall_rating]

        rating += 1
    
    page += 1
    driver.execute_script("window.history.go(-1)")
    driver.implicitly_wait(3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
CPU times: user 1min 15s, sys: 2.5 s, total: 1min 18s
Wall time: 18min 25s


In [ ]:
def clean_review(s):
    string = s.splitlines()
    string = list(filter(None, string))
    del string[0:2]
    del string[-2:]
    return string

In [ ]:
all_comments["clean_review"] = all_comments["product_review"].map(clean_review)

In [ ]:
all_comments.shape

(6205, 5)

In [ ]:
all_comments.head()

,Product_name,product_review,user_ratings,overall_rating,clean_review
0,Kentucky Brunch Brand Stout,HOPTOMIC_BOMB from California\n\n5/5 rDev +2%...,+2%,5,[look: 5 | smell: 5 | taste: 5 | feel: 5 | ove...
1,Kentucky Brunch Brand Stout,Prospero from Colorado\n\n4.53/5 rDev -7.6%\n...,-7.6%,4.53,[look: 4.5 | smell: 4.5 | taste: 4.5 | feel: 4...
2,Kentucky Brunch Brand Stout,BourbonForBeer from Illinois\n\n1.49/5 rDev -...,-69.6%,1.49,[look: 3 | smell: 1.5 | taste: 1.5 | feel: 1.5...
3,Kentucky Brunch Brand Stout,JellybonesElmer from Wisconsin\n\n4.52/5 rDev...,-7.8%,4.52,[look: 4 | smell: 4.5 | taste: 4.5 | feel: 5 |...
4,Kentucky Brunch Brand Stout,WormGod from New York\n\n4.99/5 rDev +1.8%\nl...,+1.8%,4.99,[look: 4.75 | smell: 5 | taste: 5 | feel: 5 | ...


In [ ]:
beer_list[249]

'Flora - Blueberry'

In [ ]:
all_comments.to_csv("6205 beer reviews of 250 beers.csv", sep="\t")